In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn import svm
%matplotlib inline

Get metrics collection

In [116]:
keyboard_data = pd.read_csv('Dmitry.csv')
hacker_keyboard_data = pd.read_csv('Hacker.csv')

hatString = 'qw po as lk zx ,m we sd xc iu jh nb'


Process the data

In [117]:
keyboard_data['timestamp'] = pd.to_numeric(keyboard_data['timestamp'])
keyboard_data['keyDownTimestamp'] = pd.to_numeric(keyboard_data['keyDownTimestamp'])
keyboard_data['keyUpTimestamp'] = pd.to_numeric(keyboard_data['keyUpTimestamp'])

hacker_keyboard_data['timestamp'] = pd.to_numeric(hacker_keyboard_data['timestamp'])
hacker_keyboard_data['keyDownTimestamp'] = pd.to_numeric(hacker_keyboard_data['keyDownTimestamp'])
hacker_keyboard_data['keyUpTimestamp'] = pd.to_numeric(hacker_keyboard_data['keyUpTimestamp'])


keyboard_data.head(10)

,input,key,keyDownTimestamp,keyUpTimestamp,keypressDuration,login,timestamp,type
0,lefthand_chars_area,q,1556795140932,1556795141108,176,Dmitry,1556795140932,keydown
1,lefthand_chars_area,w,1556795141035,1556795141195,160,Dmitry,1556795141035,keydown
2,lefthand_chars_area,,1556795141163,1556795141300,137,Dmitry,1556795141163,keydown
3,lefthand_chars_area,p,1556795141628,1556795141740,112,Dmitry,1556795141628,keydown
4,lefthand_chars_area,o,1556795141707,1556795141840,133,Dmitry,1556795141707,keydown
5,lefthand_chars_area,,1556795141947,1556795142044,97,Dmitry,1556795141947,keydown
6,lefthand_chars_area,a,1556795142564,1556795142724,160,Dmitry,1556795142564,keydown
7,lefthand_chars_area,s,1556795142652,1556795142796,144,Dmitry,1556795142652,keydown
8,lefthand_chars_area,,1556795142740,1556795142867,127,Dmitry,1556795142740,keydown
9,lefthand_chars_area,l,1556795143684,1556795143787,103,Dmitry,1556795143684,keydown


In [135]:
columns=[
                                    'combination',
                                    'type',
                                    'lineChar1', 
                                    'lineChar2',
                                   'interval',
                                     'dwellChar1',
                                     'dwellChar2',
                                     'latency',
                                     'flight',
                                     'up2up']

firstLine = ['q','w','e','r','t','y','u','i','o','p']
secondLine = ['a','s','d','f','g','h','j','k','l']
thirdLine = ['z','x','c','v','b','n','m']

def prepareFirstTwo(data):
    charCombinations = pd.DataFrame(columns= columns)
    isWordStarted = False
    for i in range(0, len(data)-1):
        if (i == len(data)-1):
            continue
        char = data.iloc[i]
        nextChar = data.iloc[i+1]
        
    #     Split text by words..Words for 2-chars combination..Get assignature of first 2-chars in every word combination
        if ((char.key == ' ')):
            isWordStarted = False
            continue
        if (nextChar.key == ' '):
            continue;
        if (isWordStarted):
            continue
        
        interval = nextChar.keyDownTimestamp - char.keyUpTimestamp
        dwellChar1 = char.keyUpTimestamp - char.keyDownTimestamp
        dwellChar2 = nextChar.keyUpTimestamp - nextChar.keyDownTimestamp 
        latency = nextChar.keyUpTimestamp - char.keyDownTimestamp 
        flight = nextChar.keyDownTimestamp - char.keyDownTimestamp
        up2up = nextChar.keyUpTimestamp - char.keyUpTimestamp
        chars = char.key+nextChar.key
        if (char.key in firstLine):
            lineChar1 = 1
        elif (char.key in secondLine):
            lineChar1 = 2
        else:
            lineChar1 = 3
        if (nextChar.key in firstLine):
            lineChar2 = 1
        elif (nextChar.key in secondLine):
            lineChar2 = 2
        else:
            lineChar2 = 3  
        typeChar = char.input
        charCombinations = charCombinations.append(pd.DataFrame([[
                                                    chars, 
                                                    typeChar,
                                                    lineChar1,
                                                    lineChar2,
                                                    interval,
                                                    dwellChar1,
                                                    dwellChar2,
                                                    latency,
                                                    flight,
                                                    up2up]], columns=columns), ignore_index=True)
        isWordStarted= True
    return charCombinations



# abnorm_keyboard_data_prepared = prepareFirstTwo(abnorm_keyboard_data)
charAssignatureCombinations = [['qw', 'as', 'zx', 'we', 'sd', 'xc'], [['po', 'lk',',m', 'iu', 'jh', 'nb']]]

keyboard_data_prepared = prepareFirstTwo(keyboard_data)
hacker_keyboard_data_prepared = prepareFirstTwo(hacker_keyboard_data)

# User. left hand. Left part of keyboard
keyboard_data_prepared_lefthand_leftkeyboard = keyboard_data_prepared.loc[(keyboard_data_prepared.type == 'lefthand_chars_area') & (keyboard_data_prepared.combination.isin(charAssignatureCombinations[0]))]
keyboard_data_prepared_lefthand_leftkeyboard = keyboard_data_prepared_lefthand_leftkeyboard.drop(['combination','type', 'lineChar1','lineChar2'], axis=1)
keyboard_data_prepared_lefthand_leftkeyboard_values = keyboard_data_prepared_lefthand_leftkeyboard.values
keyboard_data_prepared_lefthand_leftkeyboard_values = np.array(keyboard_data_prepared_lefthand_leftkeyboard_values).reshape(-1,12)

# Hacker. left hand. Left part of keyboard
hacker_keyboard_data_prepared_lefthand_leftkeyboard = hacker_keyboard_data_prepared.loc[(hacker_keyboard_data_prepared.type == 'lefthand_chars_area') & (hacker_keyboard_data_prepared.combination.isin(charAssignatureCombinations[0]))]
hacker_keyboard_data_prepared_lefthand_leftkeyboard = hacker_keyboard_data_prepared_lefthand_leftkeyboard.drop(['combination', 'type', 'lineChar1','lineChar2'], axis=1)
hacker_keyboard_data_prepared_lefthand_leftkeyboard_values = hacker_keyboard_data_prepared_lefthand_leftkeyboard.values
hacker_keyboard_data_prepared_lefthand_leftkeyboard_values = np.reshape(hacker_keyboard_data_prepared_lefthand_leftkeyboard_values, (-1, 12))

# User. Right hand. Left part of keyboard
keyboard_data_prepared_righthand_leftkeyboard = keyboard_data_prepared.loc[(keyboard_data_prepared.type == 'righthand_chars_area') & (keyboard_data_prepared.combination.isin(charAssignatureCombinations[0]))]
keyboard_data_prepared_righthand_leftkeyboard = keyboard_data_prepared_righthand_leftkeyboard.drop(['combination','type', 'lineChar1','lineChar2'], axis=1)
keyboard_data_prepared_righthand_leftkeyboard_values = keyboard_data_prepared_righthand_leftkeyboard.values
keyboard_data_prepared_righthand_leftkeyboard_values = np.reshape(keyboard_data_prepared_righthand_leftkeyboard_values, (-1, 12))


# Hacker. Right hand. Left part of keyboard
hacker_keyboard_data_prepared_righthand_leftkeyboard = hacker_keyboard_data_prepared.loc[(hacker_keyboard_data_prepared.type == 'righthand_chars_area') & (hacker_keyboard_data_prepared.combination.isin(charAssignatureCombinations[0]))]
hacker_keyboard_data_prepared_righthand_leftkeyboard = hacker_keyboard_data_prepared_righthand_leftkeyboard.drop(['combination', 'type', 'lineChar1','lineChar2'], axis=1)
hacker_keyboard_data_prepared_righthand_leftkeyboard_values = hacker_keyboard_data_prepared_righthand_leftkeyboard.values
hacker_keyboard_data_prepared_righthand_leftkeyboard_values = np.reshape(hacker_keyboard_data_prepared_righthand_leftkeyboard_values, (-1, 12))



[[-73 176 160 263 103 87 -72 160 144 232 88 72]
 [-41 160 120 239 119 79 -56 176 111 231 120 55]
 [-49 152 112 215 103 63 -56 135 129 208 79 73]
 [-32 143 113 224 111 81 -80 160 175 255 80 95]
 [-66 153 128 215 87 62 -71 176 159 264 105 88]
 [-48 144 136 232 96 88 -69 157 133 221 88 64]
 [-120 191 209 280 71 89 -56 176 153 273 120 97]
 [-64 160 155 251 96 91 -72 160 144 232 88 72]
 [-80 176 176 272 96 96 -80 175 136 231 95 56]
 [-120 191 209 280 71 89 -56 176 153 273 120 97]
 [-64 160 155 251 96 91 -72 160 144 232 88 72]
 [-80 176 176 272 96 96 -80 175 136 231 95 56]]


In [137]:
model = svm.OneClassSVM(kernel='rbf', gamma=0.07, nu=0.5)
model.fit(keyboard_data_prepared_lefthand_leftkeyboard_values)
print('-1 means that this is not a user assignature')
print('user, left hand, left part of keyboard- ',model.predict(keyboard_data_prepared_lefthand_leftkeyboard_values))
print('hacker, left hand, left part of keyboard- ',model.predict(hacker_keyboard_data_prepared_lefthand_leftkeyboard_values))

model.fit(keyboard_data_prepared_righthand_leftkeyboard_values)

print('user, right hand, left part of keyboard- ',model.predict(keyboard_data_prepared_righthand_leftkeyboard_values))
print('hacker, right hand, left part of keyboard- ',model.predict(hacker_keyboard_data_prepared_righthand_leftkeyboard_values))



# -1 - identified user as hacker

-1 means that this is not a user assignature
user, left hand, left part of keyboard-  [ 1  1  1 -1 -1 -1  1  1  1  1  1  1]
hacker, left hand, left part of keyboard-  [-1 -1 -1]
user, right hand, left part of keyboard-  [ 1  1  1 -1 -1 -1  1  1  1  1  1  1]
hacker, right hand, left part of keyboard-  [-1 -1 -1]


[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]


In [ ]:
#Keystroke_SVM.py
 
from sklearn.svm import OneClassSVM
 
class SVMDetector:
#training() and testing() change, rest all remains same.
    def training(self):
        self.clf = OneClassSVM(kernel='rbf',gamma=26)
        self.clf.fit(self.train)
  
    def testing(self):
        self.u_scores = -self.clf.decision_function(self.test_genuine)
        self.i_scores = -self.clf.decision_function(self.test_imposter)
        self.u_scores = list(self.u_scores)
        self.i_scores = list(self.i_scores)

In [ ]:
#keystroke_GMM.py
 
from sklearn.mixture import GMM
import warnings
warnings.filterwarnings("ignore")
 
class GMMDetector:
 
    def training(self):
        self.gmm = GMM(n_components = 2, covariance_type = 'diag', 
                        verbose = False )
        self.gmm.fit(self.train)
  
    def testing(self):
        for i in range(self.test_genuine.shape[0]):
            j = self.test_genuine.iloc[i].values
            cur_score = self.gmm.score(j)
            self.user_scores.append(cur_score)
  
        for i in range(self.test_imposter.shape[0]):
            j = self.test_imposter.iloc[i].values
            cur_score = self.gmm.score(j)
            self.imposter_scores.append(cur_score)
